In [1]:
!pip install sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 22.2 MB/s eta 0:00:00


In [2]:
!pip show sentencepiece

Name: sentencepiece
Version: 0.1.99
Summary: SentencePiece python wrapper
Home-page: https://github.com/google/sentencepiece
Author: Taku Kudo
Author-email: taku@google.com
License: Apache
Location: /usr/local/lib/python3.10/dist-packages
Requires: 
Required-by: 


**sentencepiece is needed for the colosal LLaMA**

In [3]:
!pip install -q accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 transformers==4.31.0 trl==0.4.7

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.5/92.5 MB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 115.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.4/77.4 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 80.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 40.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.9 MB/s eta 0:00:00


In [4]:
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer
from datasets import Dataset

In [5]:
from transformers import PushToHubCallback
from huggingface_hub import HfFolder
import sys
from huggingface_hub import hf_hub_download

In [6]:
from huggingface_hub import notebook_login

notebook_login()

In [7]:
from datasets import load_dataset
from sklearn.model_selection import train_test_split

Asamaa = load_dataset("SamaaMaged-205/Tech_M_asamaa")

train_data, test_data = train_test_split(Asamaa['train'], test_size=0.2, random_state=42)


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [8]:
train_data

{'merged_column': ['<s>[INST] هل من الضروري تغطية الطعام أثناء الطهي في الميكروويف ، وإذا كان الأمر كذلك ، فما هي المواد الآمنة للاستخدام؟ [/INST] نعم ، يوصى بتغطية الطعام أثناء الطهي في الميكروويف لفخار الحرارة والبخار ، مما يساعد الطعام على الطهي بسرعة أكبر.تشمل المواد الآمنة لتغطية الطعام في الميكروويف غلاف بلاستيكي آمن من الميكروويف والأواني الزجاجية الآمنة من الميكروويف والمناشف الورقية الآمنة من الميكروويف أو المناديل.من المهم تجنب استخدام المواد المعدنية أو المعدنية لتغطية الطعام في الميكروويف.</s>',
  '<s>[INST] هل من الضروري تغطية الطعام أثناء الطهي في الميكروويف ، وإذا كان الأمر كذلك ، فما هي المواد الآمنة للاستخدام؟ [/INST] يمكن أن يساعد تغطية الطعام أثناء الطهي في الميكروويف في فخ الحرارة والبخار ، مما يتسبب في طهي الطعام بسرعة أكبر.تشمل المواد الآمنة التي يجب استخدامها لتغطية أغطية آمنة من الميكروويف ، وفيلم تشبث بالميكروويف مع زاوية مطوية ، وورق مقاوم للاسترداد.ومع ذلك ، من المهم أن نلاحظ أن التغطية مع ورق شحوم قد يتسبب في جفاف الطعام قليلاً.</s>',
  '<s>[INST] هل يمكنك ت

In [10]:
model_name = "hpcai-tech/Colossal-LLaMA-2-7b-base"

# Fine-tuned model name
new_model = "llama-2-7b-miniguanaco"

# QLoRA parameters

# LoRA attention dimension
lora_r = 64

# Alpha parameter for LoRA scaling
lora_alpha = 16

# Dropout probability for LoRA layers
lora_dropout = 0.1

# bitsandbytes parameters


# Activate 4-bit precision base model loading
use_4bit = True

# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "float16"

# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"

# Activate nested quantization for 4-bit base models (double quantization)
use_nested_quant = False

# TrainingArguments parameters

# Output directory where the model predictions and checkpoints will be stored
output_dir = "./results"

# Number of training epochs
num_train_epochs = 1

# Enable fp16/bf16 training (set bf16 to True with an A100)
fp16 = False
bf16 = False

# Batch size per GPU for training
per_device_train_batch_size = 4

# Batch size per GPU for evaluation
per_device_eval_batch_size = 4

# Number of update steps to accumulate the gradients for
gradient_accumulation_steps = 1

# Enable gradient checkpointing
gradient_checkpointing = True

# Maximum gradient normal (gradient clipping)
max_grad_norm = 0.3

# Initial learning rate (AdamW optimizer)
learning_rate = 2e-4

# Weight decay to apply to all layers except bias/LayerNorm weights
weight_decay = 0.001

# Optimizer to use
optim = "paged_adamw_32bit"

# Learning rate schedule (constant a bit better than cosine)
lr_scheduler_type = "constant"

# Number of training steps (overrides num_train_epochs)
max_steps = -1

# Ratio of steps for a linear warmup (from 0 to learning rate)
warmup_ratio = 0.03

# Group sequences into batches with same length
# Saves memory and speeds up training considerably
group_by_length = True

# Save checkpoint every X updates steps
save_steps = 75

# Log every X updates steps
logging_steps = 25


# SFT parameters


# Maximum sequence length to use
max_seq_length = 269

# Pack multiple short examples in the same input sequence to increase efficiency
packing = False

# Load the entire model on the GPU 0
device_map = {"": 0}

In [11]:


# Load tokenizer and model with QLoRA configuration
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

# Check GPU compatibility with bfloat16
if compute_dtype == torch.float16 and use_4bit:
    major, _ = torch.cuda.get_device_capability()
    if major >= 8:
        print("=" * 80)
        print("Your GPU supports bfloat16: accelerate training with bf16=True")
        print("=" * 80)

# Load base model
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map=device_map
)
model.config.use_cache = False
model.config.pretraining_tp = 1

# Load LLaMA tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right" # Fix weird overflow issue with fp16 training

# Load LoRA configuration
peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
)

config.json:   0%|          | 0.00/600 [00:00<?, ?B/s]

pytorch_model.bin.index.json:   0%|          | 0.00/21.3k [00:00<?, ?B/s]

pytorch_model-00001.bin:   0%|          | 0.00/1.07G [00:00<?, ?B/s]

pytorch_model-00002.bin:   0%|          | 0.00/1.02G [00:00<?, ?B/s]

pytorch_model-00003.bin:   0%|          | 0.00/1.03G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/171 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/745 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/1.05M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/411 [00:00<?, ?B/s]

You are using the legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This means that tokens that come after special tokens will not be properly handled. We recommend you to read the related pull request available at https://github.com/huggingface/transformers/pull/24565


In [17]:


# Assuming 'train' is the split you are interested in
# Replace 'train' with the actual split name if different
text_lengths = []

# Iterate over the dataset and calculate the length of each text
for example in Asamaa['train']:
    text = example['merged_column']  # Replace with the actual text column name
    text_length = len(text.split())  # Assuming words are separated by spaces
    text_lengths.append(text_length)

# Calculate the maximum length
max_length = max(text_lengths)

print(f"The maximum text length is: {max_length} words")


The maximum text length is: 269 words


In [12]:
# Set training parameters
training_arguments = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    fp16=fp16,
    bf16=bf16,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=group_by_length,
    lr_scheduler_type=lr_scheduler_type,
    report_to="tensorboard",

)


In [20]:
print(Asamaa.column_names)


{'train': ['merged_column']}


In [51]:
from sklearn.metrics import accuracy_score
def custom_evaluation_metric(predictions, labels):
    # Implement your custom evaluation metric
    # This is just an example using accuracy as a placeholder
    accuracy = accuracy_score(predictions, labels)

In [52]:
# Set supervised fine-tuning parameter
trainer = SFTTrainer(
    model=model,
    train_dataset=Asamaa['train'],  # Access the 'train' key
    peft_config=peft_config,
    dataset_text_field="merged_column",  # Use the correct column name
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
    packing=packing,
    compute_metrics=custom_evaluation_metric,

)


/usr/local/lib/python3.10/dist-packages/peft/utils/other.py:102: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(


Map:   0%|          | 0/378 [00:00<?, ? examples/s]

In [21]:
# Set supervised fine-tuning parameters
trainer = SFTTrainer(
    model=model,
    train_dataset=Asamaa['train'],  # Access the 'train' key
    peft_config=peft_config,
    dataset_text_field="merged_column",  # Use the correct column name
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
    packing=packing,
)


Map:   0%|          | 0/378 [00:00<?, ? examples/s]

In [22]:
# Train model
trainer.train()

You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
25,1.371700
50,0.988800
75,0.776000


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


TrainOutput(global_step=95, training_loss=0.975701904296875, metrics={'train_runtime': 414.0082, 'train_samples_per_second': 0.913, 'train_steps_per_second': 0.229, 'total_flos': 2146597495603200.0, 'train_loss': 0.975701904296875, 'epoch': 1.0})

In [23]:
trainer.model.save_pretrained(new_model)


In [40]:
from nltk.translate.bleu_score import sentence_bleu
import re


In [38]:
def compute_metrics_bleu(pred):
    references = pred.label_ids
    generated_texts = pred.predictions
    print("references:",references)
    print("generated_texts:",generated_text)

    bleu_scores = []
    for reference, generated_text in zip(references, generated_texts):
        reference_text = train_data[reference]['text']
        bleu_score = sentence_bleu([reference_text], generated_text)
        bleu_scores.append(bleu_score)

    return {
        'bleu': sum(bleu_scores) / len(bleu_scores)
    }

In [45]:
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200)
# result = pipe(f"<s>[INST] {questions} [/INST]")
# print(result[0]['generated_text'])

def preprocess_train_data(data):
    questions = []
    answers = []

    for entry in data:
        match = re.search(r'<s>\[INST\](.*?)\[/INST\]', entry)
        if match:
            question = match.group(1).strip()
            answer = entry.split('[/INST]')[-1].strip()
            questions.append(question)
            answers.append(answer)
        else:
            print(f"No match found in the training entry: {entry}")

    return questions, answers


# Preprocess train data
questions, actual_answers = preprocess_train_data(train_data)

# Initialize an empty list to store generated responses
generated_responses = []

# Run text generation pipeline with your trained model on each question
# Run text generation pipeline with your trained model on each question
for question in questions:
    result = pipe(f"<s>[INST] {question} [/INST]")
    #result = pipe(f"<s>[INST] {questions} [/INST]")
    #result =  result[0]['generated_text']
    if 'generated_text' in result[0]:
        generated_responses.append(result[0]['generated_text'])
    else:
        print(f"Error: 'generated_text' not found in result: {result}")


# Create a dummy class for predictions with label_ids and predictions attributes
class DummyPrediction:
    def __init__(self, label_ids, predictions):
        self.label_ids = label_ids
        self.predictions = predictions

# Create an instance of the dummy class with label_ids and generated_responses
dummy_pred = DummyPrediction(list(range(len(generated_responses))), generated_responses)

# Use the compute_metrics_bleu function
bleu_metrics = compute_metrics_bleu(dummy_pred)

# Print or use the BLEU score for evaluation
print(f"BLEU Score: {bleu_metrics['bleu']}")

No match found in the training entry: merged_column
references: []


UnboundLocalError: ignored

In [26]:
# Ignore warnings
logging.set_verbosity(logging.CRITICAL)

# Run text generation pipeline with our next model
prompt = "ما هي خطوات تشغيل الميكروويف؟"
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200)
result = pipe(f"<s>[INST] {prompt} [/INST]")
print(result[0]['generated_text'])

<s>[INST] ما هي خطوات تشغيل الميكروويف؟ [/INST] لتشغيل الميكروويف ، اتبع هذه الخطوات:
1. اضغط على زر المشغل.
2. اضغط على زر المشغل مرة أخرى لتشغيل الميكروويف.
3. اضغط على زر المشغل مرة أخرى لتشغيل الميكروويف.
4. اضغط على زر المشغل مرة أخرى لتشغيل الميكروويف.
5. اضغط على زر المشغل مرة أخرى لتشغيل المي


In [ ]:
# Ignore warnings
logging.set_verbosity(logging.CRITICAL)

# Run text generation pipeline with our next model
prompt = "هل هناك أي إرشادات لتسخين السوائل في الميكروويف؟"
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200)
result = pipe(f"<s>[INST] {prompt} [/INST]")
print(result[0]['generated_text'])

<s>[INST] هل هناك أي إرشادات لتسخين السوائل في الميكروويف؟ [/INST] نعم ، يوفر الميكروويف إرشادات لتسخين السوائل في الميكروويف.يمكن أن تسخين السوائل من أجل التسخين أو التسخين مع أي ورق من الميكروويف ، ومن المهم أن تستخدم الميكروويف


In [ ]:
# Ignore warnings
logging.set_verbosity(logging.CRITICAL)

# Run text generation pipeline with our next model
prompt = "كيف  أشغل الميكروويف؟"
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200)
result = pipe(f"<s>[INST] {prompt} [/INST]")
print(result[0]['generated_text'])

/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:61: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


<s>[INST] كيف  أشغل الميكروويف؟ [/INST] لشغل الميكروويف ، اضغط على زر البدء أو اضغط على زر البدء في القائمة الموجودة في الميكروويف.من المهم اتباع بيندراج الميكروويف للحصول على مستوى الطاقة الموجود في الميكروويف.إذا كان الميكروويف مستوي


In [ ]:
# Ignore warnings
logging.set_verbosity(logging.CRITICAL)

# Run text generation pipeline with our next model
prompt = "ما هي خطوات تشغيل الميكروويف؟"
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200)
result = pipe(f"<s>[INST] {prompt} [/INST]")
print(result[0]['generated_text'])

<s>[INST] ما هي خطوات تشغيل الميكروويف؟ [/INST] لتشغيل الميكروويف ، اتبع هذه الخطوات:
1. اضغط على زر المستوى لتحديد المستوى المطلوب.
2. اضغط على زر البدء لبدء عملية التشغيل.
3. اضغط على زر المستوى لتغلية المستوى في مستوى مطلوب.



In [ ]:


# Reload model in FP16 and merge it with LoRA weights
base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map=device_map,
)
model = PeftModel.from_pretrained(base_model, new_model)
model = model.merge_and_unload()

# Reload tokenizer to save it
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

Loading checkpoint shards:   0%|          | 0/14 [00:00<?, ?it/s]

OutOfMemoryError: ignored